In [1]:
import pandas as pd
from timeseries_utils import read_daily_timeseries_csv

import pathlib

import warnings
warnings.filterwarnings("error")

# The purpose of this script is to make the timeseries organised by attribute to the same format as the ones organised by gauge.
It also drops some catchments that were discarded after the attribute was calculated

In [2]:
dst_dir = '/media/iielse/T9/CAMELS-FI/data/timeseries_by_attribute'
src_dir = '/home/iielse/Documents/DIWA/data/CAMELS-FI construction/timeseries_by_attribute'

# Used to get the definitive final set of gauges
gauge_meta_path = '/home/iielse/Documents/DIWA/data/CAMELS-FI construction/CAMELS_FI_meta_attributes.csv'
meta = pd.read_csv(gauge_meta_path)



root_path = pathlib.Path(src_dir)
# Getting the files in 
files = [item for item in root_path.iterdir() if item.is_file()]

In [3]:
for file in files:
    # some files ended up with format ,"0.5", "0.1" that needs changing
    comma_files = ['discharge_spec', 'discharge_vol', 'humidity_rel',
                   'pe_era5_land', 'pet', 'pet_fmi', 
                  ]
    if file.stem in comma_files:
        timeseries = read_daily_timeseries_csv(file, decimal=',')
    else:
        timeseries = read_daily_timeseries_csv(file)
    
    # Renaming virtual gauges to use '-' as a separator instead of '_'this caused some bugs as _ was used in file names
    renames = {name:name.replace('_', '-') for name in timeseries.columns if '_' in name}
    timeseries = timeseries.rename(renames, axis=1)
    # Selecting only the final gauges
    timeseries = timeseries[meta.gauge_id]
    dst_path = pathlib.Path(dst_dir).joinpath(file.name)
    timeseries.to_csv(dst_path)